# Task 3 model implementation

In [ ]:
import numpy as np
import pandas as pd

from model import Model, Optimizer, NetworkModification, TransitScheduleModification

Creating a model implementation only requires pointing to a folder with all the necessary input data. Here, we have a region with 4 microtypes and four modes (walk, bus, auto, and rail), with rail only service two of the four microtypes

Once the model is created, you find supply/demand equilibrium and gather statistics such as mode splits, speeds, and user costs for an individual time period or for the whole day.

In [ ]:
model = Model("input-data")

userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

Once a model is created, there are a set of objects and methods that make it easy to modify the transportation system without needing to change the original input files.

Here, we will devote some road space to dedicated bus lanes. In particular, we are transitioning 1000 lane-meters of roadway from mixed bus/car traffic to bus-only lanes in microtypes A and B:

In [ ]:
networkModification = NetworkModification(np.array([1000., 1000.]), list(zip([2, 4], [13, 14])))
model.modifyNetworks(networkModification = networkModification)

Once these changes are made, we can re-equilibrate the model and look at how metrics have changed. Here, we have (slightly) increased the bus mode share, in large part by increasing average bus speed in Microtypes A and B. However, because we have decreased auto speeds in these two microtypes, total user costs have gone up.

In [ ]:
userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

You can also modify the headways for the different transit modes. Here, we can decrease the headway of the bus mode in Microtype A to 200 seconds.

In [ ]:
scheduleModification = TransitScheduleModification([200.0], [("A", "bus")])
model.modifyNetworks(scheduleModification = scheduleModification)
userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

Here, we can see that the bus mode share went up (because of lower wait times). In addition, bus speeds in microtype A increased because there was less crowding on each bus, but bus speeds slightly decreased in the other microtypes because of the increase in bus ridership without a corresponding decrease in headways.

In [ ]:
print(model.getModeSpeeds('Overnight'))

In [ ]:
    o = Optimizer("input-data",
                  modesAndMicrotypes=list(zip(["A", "B", "C", "D"], ["bus", "bus", "bus", "bus"])),
                  method="noisy")
    output = o.minimize()